In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

print(mean)
print(std)

120.70748
64.150024


In [ ]:
weight_decay = 1e-4
model = Sequential()
model.add(Lambda(lambda x: (x-120.70748)/64.150024, input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)       

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [ ]:
#training
batch_size = 64
 
opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
781/781 [==============================] - 28s 34ms/step - loss: 2.3611 - accuracy: 0.3422 - val_loss: 1.3902 - val_accuracy: 0.5769
Epoch 2/125
781/781 [==============================] - 26s 33ms/step - loss: 1.3268 - accuracy: 0.5688 - val_loss: 1.0623 - val_accuracy: 0.6747
Epoch 3/125
781/781 [==============================] - 26s 33ms/step - loss: 1.0956 - accuracy: 0.6449 - val_loss: 0.9292 - val_accuracy: 0.7194
Epoch 4/125
781/781 [==============================] - 26s 33ms/step - loss: 0.9848 - accuracy: 0.6858 - val_loss: 0.9175 - val_accuracy: 0.7291
Epoch 5/125
781/781 [==============================] - 26s 33ms/step - loss: 0.9094 - accuracy: 0.7192 - val_loss: 0.8532 - val_accuracy: 0.7468
Epoch 6/125
781/781 [==============================] - 26s 33ms/step - loss: 0.8645 - accuracy: 0.7346 - val_loss: 0.7582 - val_accuracy: 0.7801
Epoch 7/125
781/781 [==============================] - 26s 33ms/step - loss: 0.8178 - accuracy: 0.7544 - val_loss: 0.8696 - val_ac

In [ ]:
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

79/79 [==============================] - 1s 6ms/step - loss: 0.4499 - accuracy: 0.8890

Test result: 88.900 loss: 0.450


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
export_path="model.h5"
model.save(export_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import load_model
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
model = load_model("model.h5")
scores = model.evaluate(X_test, to_categorical(Y_test))
model.summary()
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

170500096/170498071 [==============================] - 13s 0us/step


OSError: ignored

In [ ]:
!zip -r model.zip {export_path}

updating: model.h5 (deflated 13%)


In [ ]:
try:
    from google.colab import files
    files.download('./model.zip')
except ImportError:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>